In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

In [23]:
all_imgs["path"].str.split("/").str[-2].value_counts()

train    5778
test     1656
val       826
Name: path, dtype: int64

In [14]:
from tqdm import tqdm

In [15]:

N_FOLDS = 5

my_path = "../datasets/kneeKL224_all/"
shutil.rmtree(my_path)

os.makedirs(my_path, exist_ok=True)

all_imgs = []

for dirpath, dirnames, filenames in os.walk("../datasets/kneeKL224/"):
    if "auto_test" in dirpath:
        continue
    else:
        if dirpath[-1].isnumeric():
            for filename in filenames:
                all_imgs.append([dirpath, filename])

all_imgs = pd.DataFrame(all_imgs, columns=["path", "filename"])
all_imgs["class_id"] = all_imgs["path"].str[-1]
all_imgs["full_path"] = (all_imgs["path"]+"/"+all_imgs["filename"])

for fold_no in range(N_FOLDS):
    path_to_fold_number = f"../datasets/kneeKL224_all/fold{fold_no}"
    os.makedirs(path_to_fold_number, exist_ok=True)
    for phase in ["train", "test", "val"]:
        for class_no in range(5):
            os.makedirs(f"{path_to_fold_number}/{phase}/{class_no}", exist_ok=True)

    train, val = train_test_split(all_imgs, stratify=all_imgs["class_id"], train_size=0.7, random_state=fold_no)
    val, test = train_test_split(val, stratify=val["class_id"], train_size=0.666666, random_state=fold_no)

    train[f"phase_in_fold_{fold_no}"] = "train"
    val[f"phase_in_fold_{fold_no}"] = "val"
    test[f"phase_in_fold_{fold_no}"] = "test"
    fold_info = pd.concat([train, val, test])[f"phase_in_fold_{fold_no}"]
    all_imgs = all_imgs.join(fold_info)    


for _, s in tqdm(all_imgs.iterrows()):
    src = s["full_path"]
    filename = s["filename"]
    class_no = s["class_id"]

    for fold_no in range(N_FOLDS):
        phase = s[f"phase_in_fold_{fold_no}"]
        
        dst = f"../datasets/kneeKL224_all/fold{fold_no}/{phase}/{class_no}/{filename}"
        shutil.copy(src=src, dst=dst)

8260it [03:54, 35.21it/s]


In [5]:
tqdm(total = len(all_imgs))

,path,filename,class_id,full_path,phase_in_fold_0,phase_in_fold_1,phase_in_fold_2,phase_in_fold_3,phase_in_fold_4
0,../datasets/kneeKL224/train/1,9853562R.png,1,../datasets/kneeKL224/train/1/9853562R.png,train,train,train,val,test
1,../datasets/kneeKL224/train/1,9306384R.png,1,../datasets/kneeKL224/train/1/9306384R.png,train,train,train,train,val
2,../datasets/kneeKL224/train/1,9463326L.png,1,../datasets/kneeKL224/train/1/9463326L.png,test,train,train,val,train
3,../datasets/kneeKL224/train/1,9943227L.png,1,../datasets/kneeKL224/train/1/9943227L.png,train,val,train,val,train
4,../datasets/kneeKL224/train/1,9800285L.png,1,../datasets/kneeKL224/train/1/9800285L.png,train,train,train,val,train
...,...,...,...,...,...,...,...,...,...
8255,../datasets/kneeKL224/test/0,9024409R.png,0,../datasets/kneeKL224/test/0/9024409R.png,train,train,val,train,train
8256,../datasets/kneeKL224/test/0,9532148R.png,0,../datasets/kneeKL224/test/0/9532148R.png,train,train,train,test,train
8257,../datasets/kneeKL224/test/0,9598675L.png,0,../datasets/kneeKL224/test/0/9598675L.png,val,train,train,train,train
8258,../datasets/kneeKL224/test/0,9338346L.png,0,../datasets/kneeKL224/test/0/9338346L.png,train,train,train,test,test
